https://towardsdatascience.com/computer-vision-for-beginners-part-1-7cca775f58ef

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# OpenCV color standard is BGR (Blue, Green, Red)
# So when the image is displayed, it'll look kinda funny until you convert it to RGB
img = cv2.imread('Pepsi_logo_2014.png')
print(cv2.__version__)
type(img)

In [ ]:
# All OpenCV color transformations
print([x for x in dir(cv2) if x.startswith('COLOR_')])

In [ ]:
# simple function to return heigh/width/depth; frequently used throughout for verification
def image_dimensions(img):
    img_dim = ('Height:','Width: ','Depth: ')
    
    for i in range(img.ndim):
        print(img_dim[i],img.shape[i])

In [ ]:
# split and display each color individually; not really sure if it will be important going forward, but just in case
def split_color_display (img):
    # Plot the three channels of the image
    fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize = (20, 20))

    for i in range(0, 3):
        ax = axs[i]
        ax.imshow(cv2.split(img)[i])
    plt.show()

How-To Blend Images: https://docs.opencv.org/3.4/d5/dc4/tutorial_adding_images.html

In [ ]:
# blends two images together; essentially adding both their matrices
def blend_images (src1, src2, alpha):
    
    height_1, width_1, depth_1 = src1.shape # get shape of image 1
    height_2, width_2, depth_2 = src2.shape # get shape of image 2
    src3 = cv2.resize(src2, (width_1,height_1)) # resize image 2 to match image 1
    
    # [blend_images]
    beta = (1.0 - alpha) # alpha/beta is % of blend. higher alpha = more of image 1, lower alpha = more of image 2
    new_img = cv2.addWeighted(src1, alpha, src3, beta, 0.0)
    
    return new_img

In [ ]:
# measures two images, resizes 'source' image to match the size of the 'destination' image
def resize_image (destination, source):
    
    height_1, width_1, depth_1 = destination.shape # get shape of image 1
    height_2, width_2, depth_2 = source.shape # get shape of image 2
    
    if (height_1==height_2 & width_1==width_2):
        target = source
    else:
        target = cv2.resize(source, (width_1,height_1)) # resize image 2 to match image 1
        
    return target

In [ ]:
# rotates an image counter-clockwise by [degrees]
# [shrinkage] is a percentage that shrinks the image (to ensure it stays within the image dimension bounds)
# eg. rotate_image(img, 45, 0.7) will take the [img], 
# rotate it 45 degrees counter-clockwise and shrink it to 70% of its orignal size
def rotate_image (img, degrees, shrinkage):
    num_rows, num_cols = img.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), degrees, shrinkage)
    rotated_img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
    return rotated_img

In [ ]:
# Convert img from BGR to RGB
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image_dimensions(rgb_img) # print image dimensions

plt.title('RGB Image')
plt.imshow(rgb_img)

split_color_display(rgb_img) # split all three colors into separate plots

In [ ]:
# Convert img from BGR to GRAYSCALE
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

image_dimensions(gray_img)

plt.title('Gray Image')
plt.imshow(gray_img, cmap='gray')

In [ ]:
# Convert img from BGR to GRAYSCALE
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
image_dimensions(hsv_img)

plt.title('HSV Image')
plt.imshow(hsv_img)

split_color_display(hsv_img)

In [ ]:
# combine pepsi logo with crumpled paper
img1 = rgb_img
img2 = cv2.imread('crumpled-paper.jpg')

print('Image to be merged')
plt.figure()
plt.imshow(img2)
plt.show()

new_img2 = resize_image(img1,img2)
blended_image = blend_images(img1,new_img2,.5)
image_dimensions(blended_image)

print('Merged Image')
plt.figure()
plt.imshow(blended_image)
plt.show()

In [ ]:
#rotate combined pepsi/paper logo
rotated_img = rotate_image(blended_image, 45, .9)
plt.title("Rotated Image")
plt.imshow(rotated_img)

Affine Transformations: https://www.cis.rit.edu/class/simg782/lectures/lecture_02/lec782_05_02.pdf

In [ ]:
# basically transforming the image without maintaining its dimensions
# 'src_points' is your dimensions of your image
# 'dst_points' is your future dimensions of your image (eg. from square to parellogram etc)
# still fiddling, trying to understand the math behind the code sample before I generalize the code

rows, cols = blended_image.shape[:2]
src_points = np.float32([[0,0], [cols-1,0], [0,rows-1]])
dst_points = np.float32([[0,0], [int(0.6*(cols-1)),0],[int(0.4*(cols-1)),rows-1]])
affine_matrix = cv2.getAffineTransform(src_points, dst_points)
img_output = cv2.warpAffine(blended_image, affine_matrix, (cols,rows))
plt.imshow(img_output)